In [3]:
import pandas as pd
from pandas.io.json import json_normalize
pd.options.display.max_rows = 999

import time
import datetime as dt
import numpy as np
import shutil
import os
from pathlib import Path
import requests
import json
import csv
import urllib3
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

#import plotly.express as px
#import plotly.graph_objs as go
#import matplotlib.pyplot as plt

note: this is how you would generate the first strava_tokens json.  you need the "code" from the first authorization attempt at this URL: https://www.strava.com/oauth/authorize?client_id=88709&response_type=code&redirect_uri=http://localhost/exchange_token&approval_prompt=force&scope=activity:read_all 

In [82]:

response = requests.post(
    url = 'https://www.strava.com/oauth/token',
    data = {
        'client_id': 88709,
        'client_secret': '54dab45b88619205ee1e47eb94d0f2d84762a4b2',
        'code': '0b5e16a33598c632504a40b8fcd2601ed865f2b4',
        'grant_type': 'authorization_code'
    }
)

#Save json response as a variable
strava_tokens = response.json()# Save tokens to file
with open('strava_tokens.json', 'w') as outfile:
    json.dump(strava_tokens, outfile)# Open JSON file and print the file contents 
# to check it's worked properly
with open('strava_tokens.json') as check:
  data = json.load(check)
print(data)

{'token_type': 'Bearer', 'expires_at': 1656238989, 'expires_in': 20126, 'refresh_token': 'f3e2ea76df5b6708b743a33144f8f6445b203dc1', 'access_token': '3391838e88c8aea3f59fe1ff06e4995bbffca9b5', 'athlete': {'id': 23871091, 'username': None, 'resource_state': 2, 'firstname': 'Daniel', 'lastname': 'Sutton', 'bio': None, 'city': 'Portland', 'state': 'Oregon', 'country': 'United States', 'sex': 'M', 'premium': True, 'summit': True, 'created_at': '2017-08-01T20:06:35Z', 'updated_at': '2022-06-25T20:31:09Z', 'badge_type_id': 1, 'weight': 92.9864, 'profile_medium': 'https://dgalywyr863hv.cloudfront.net/pictures/athletes/23871091/13279718/3/medium.jpg', 'profile': 'https://dgalywyr863hv.cloudfront.net/pictures/athletes/23871091/13279718/3/large.jpg', 'friend': None, 'follower': None}}


and here is the script to generate your strava activities

to do: 
* clean the data obviously
* make it so every new call just appends new data to the "activities.csv"

In [4]:
with open('strava_tokens.json') as json_file:
    strava_tokens = json.load(json_file)

if strava_tokens['expires_at'] < time.time():
    response = requests.post(
                        url = 'https://www.strava.com/oauth/token',
                        data = {
                                'client_id': 88709,
                                'client_secret': '54dab45b88619205ee1e47eb94d0f2d84762a4b2',
                                'grant_type': 'refresh_token',
                                'refresh_token': strava_tokens['refresh_token']
                                }
                    ) 
    new_strava_tokens = response.json() 
    with open('strava_tokens.json', 'w') as outfile:
        json.dump(new_strava_tokens, outfile)
    strava_tokens = new_strava_tokens

page = 1
url = "https://www.strava.com/api/v3/activities"
access_token = strava_tokens['access_token']

activities = pd.DataFrame()

while True:
    
    r = requests.get(url + '?access_token=' + access_token + '&per_page=200' + '&page=' + str(page))
    r = r.json()    # if no results then exit loop
    if (not r):
        break
    
    activities = pd.concat([activities, json_normalize(r)])
    
    page += 1
    
activities.to_csv('strava_activities.csv')

C:\Users\dansu\AppData\Local\Temp\ipykernel_524\1905212187.py:32: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead.
  activities = pd.concat([activities, json_normalize(r)])
C:\Users\dansu\AppData\Local\Temp\ipykernel_524\1905212187.py:32: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead.
  activities = pd.concat([activities, json_normalize(r)])
C:\Users\dansu\AppData\Local\Temp\ipykernel_524\1905212187.py:32: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead.
  activities = pd.concat([activities, json_normalize(r)])
C:\Users\dansu\AppData\Local\Temp\ipykernel_524\1905212187.py:32: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead.
  activities = pd.concat([activities, json_normalize(r)])


In [3]:
activities = pd.read_csv('strava_activities.csv')

In [4]:
activities.shape

(795, 59)

In [2]:
p = Path.home()
cwd = Path.cwd()
readme = cwd / 'README.md'
activities_csv_downloaded = str(p / 'Downloads' / 'Activities.csv')
activities_csv_raw = str(cwd / 'data' / '01_raw' / 'Activities.csv')
activities_csv_processed = str(cwd / 'data' / '02_processed' / 'Activities_processed.csv')

In [5]:
# activities_df_downloaded = pd.read_csv(activities_csv_downloaded)
# activities_df_raw = pd.read_csv(activities_csv_raw)
# activities_df_raw = pd.concat([activities_df_downloaded, activities_df_raw]).drop_duplicates(subset='Date')
# activities_df_raw.to_csv(activities_csv_raw)
# os.remove(activities_csv_downloaded)

In [47]:
def clean_run_pace(df):
    '''
    Converts our mm:ss string to a datetime object, then back to a string but in HH:MM:SS format since
    that's what to_timedelta() takes.  

    Time deltas are for durations, while datetimes are for moments in time.      
    '''
    df.best_pace = df.best_pace.apply(lambda x: dt.datetime.strptime(x, '%M:%S'))
    df.best_pace = df.best_pace.apply(lambda x: x.strftime('%H:%M:%S'))
    df.best_pace = pd.to_timedelta(df.best_pace)
    df.avg_pace = df.avg_pace.apply(lambda x: dt.datetime.strptime(x, '%M:%S'))
    df.avg_pace = df.avg_pace.apply(lambda x: x.strftime('%H:%M:%S'))
    df.avg_pace = pd.to_timedelta(df.avg_pace)
    return df

def clean_bike_pace(df):
    df.avg_pace = df.avg_pace.astype(float)
    return df

def download_to_raw():
    activities_df_downloaded = pd.read_csv(activities_csv_downloaded)
    activities_df_raw = pd.read_csv(activities_csv_raw)
    activities_df_raw = pd.concat([activities_df_downloaded, activities_df_raw]).drop_duplicates(subset='Date')
    activities_df_raw.to_csv(activities_csv_raw, index=False)
    os.remove(activities_csv_downloaded)

def prep_df(df):
    df.columns = df.columns.str.lower().str.replace(' ', '_')
    df.drop(df.columns[17:29], axis=1, inplace=True)
    df.drop(['favorite'], axis=1, inplace=True)
    df.replace('--', '0', inplace=True)
    df.date = pd.to_datetime(df.date)
    df.set_index('date', inplace=True)
    return df

# def clean_time(time):
#     #df.time = pd.to_timedelta(df.time)
#     time = time.split('.')[0]
#     time_pieces = time.split(':')
#     if len(time_pieces) < 3:
#         time_pieces.insert(0, '00')
#     time = ':'.join(time_pieces)
#     time = dt.datetime.strptime(time, '%H:%M:%S')
#     #time = time.time
#     return time

def clean_time(df):
    #df.time = df.time.split('.')[0]
   # df.time = df.time.apply()
    df.time.apply(lambda x: (lambda y: y.split(':')).insert(0, '00') if len(x) < 3 else x)
    time_pieces = df.time.apply(lambda x: x.split(':'))
    #df.time = pd.to_timedelta(df.time)
    #time = time.split('.')[0]
    #time_pieces = time.split(':')
    if len(time_pieces) < 3:
        time_pieces.insert(0, '00')
    time = ':'.join(time_pieces)
    time = dt.datetime.strptime(time, '%H:%M:%S')
    df.time = time
    return df

def cols_to_float(df):
    df.calories = df.calories.str.replace(',', '').astype(float)
    df.iloc[:, 5:10] = df.iloc[:, 5:10].replace('--', '0')
    df.iloc[:, 5:10] = df.iloc[:, 5:10].astype(float)
    return df

def split_run_and_bike(df):
    run = df[df.activity_type == 'Running']
    bike = df[df.activity_type == 'Cycling']
    return run, bike

def clean_and_convert(df):
    df = clean_time(df)
    df = cols_to_float(df)
    return df

In [42]:
data.time[5]

'00:41:35'

In [49]:
data.columns[5:10]

Index(['avg_hr', 'max_hr', 'aerobic_te', 'avg_run_cadence', 'max_run_cadence'], dtype='object')

In [16]:
p = Path.home()
cwd = Path.cwd()

readme = str(p / 'Documents' / 'Python' / 'fitprogress' / 'README.md')
activities_csv_downloaded = str(p / 'Downloads' / 'Activities.csv')
activities_csv_raw = str(p / 'Documents' / 'Python' / 'fitprogress' / 'data' / '01_raw' / 'Activities.csv')
activities_csv_processed = str(p / 'Documents' / 'Python' / 'fitprogress' / 'data' / '02_processed' / 'Activities_processed.csv')
emojis = str(p / 'Documents' / 'Python' / 'fitprogress' / 'data' / 'emoji.json')


In [33]:
data_raw.columns[17:29]

Index(['Avg Vertical Ratio', 'Avg Vertical Oscillation',
       'Training Stress Score®', 'Grit', 'Flow', 'Dive Time', 'Min Temp',
       'Surface Interval', 'Decompression', 'Best Lap Time', 'Number of Laps',
       'Max Temp'],
      dtype='object')

In [23]:
data_raw = pd.read_csv(activities_csv_raw)

In [48]:
data = pd.read_csv(activities_csv_raw)
print(data.columns)
data = prep_df(data)
print(data.columns)
data = clean_and_convert(data)
run, bike = split_run_and_bike(data)
run = clean_run_pace(run)

Index(['Activity Type', 'Date', 'Favorite', 'Title', 'Distance', 'Calories',
       'Time', 'Avg HR', 'Max HR', 'Aerobic TE', 'Avg Run Cadence',
       'Max Run Cadence', 'Avg Speed', 'Max Speed', 'Total Ascent',
       'Total Descent', 'Avg Stride Length', 'Avg Vertical Ratio',
       'Avg Vertical Oscillation', 'Training Stress Score®', 'Grit', 'Flow',
       'Dive Time', 'Min Temp', 'Surface Interval', 'Decompression',
       'Best Lap Time', 'Number of Laps', 'Max Temp', 'Moving Time',
       'Elapsed Time', 'Min Elevation', 'Max Elevation', 'Elev Gain',
       'Elev Loss', 'Avg Pace', 'Best Pace', 'Total Strokes', 'Climb Time',
       'Bottom Time', 'Unnamed: 0', 'Unnamed: 0.1', 'Total Reps',
       'Total Sets'],
      dtype='object')
Index(['activity_type', 'title', 'distance', 'calories', 'time', 'avg_hr',
       'max_hr', 'aerobic_te', 'avg_run_cadence', 'max_run_cadence',
       'avg_speed', 'max_speed', 'total_ascent', 'total_descent',
       'avg_stride_length', 'moving_tim

TypeError: sequence item 0: expected str instance, list found

In [14]:
data.columns

Index(['activity_type', 'title', 'distance', 'calories', 'time', 'avg_hr',
       'max_hr', 'aerobic_te', 'avg_run_cadence', 'max_run_cadence',
       'avg_speed', 'max_speed', 'total_ascent', 'total_descent',
       'avg_stride_length'],
      dtype='object')

In [12]:
data = pd.read_csv(activities_csv_raw)
data = prep_df(data)
#data = clean_time(data)
data = cols_to_float(data)
#data = clean_and_convert(data)
#run, bike = split_run_and_bike(data)
#run = clean_run_pace(run)
#bike = clean_bike_pace(bike)

In [25]:
date_min_val = run.index[0].to_pydatetime()
date_max_val = run.index[-1].to_pydatetime()
date_starter_val = run.index[5].to_pydatetime()

In [61]:
run.avg_pace[2].strptime('%H:%M:%S')

AttributeError: 'Timedelta' object has no attribute 'strptime'

In [62]:
hrs, mins, secs = str(run.avg_pace[0]).split(':')

In [77]:
str(run.avg_pace.mean()).split(' ')[2].split('.')[0]

'00:09:45'

In [50]:
min, sec = str(run.avg_pace[2].seconds / 60).split('.')
min = min + ' minutes, '
sec = str(round(float('.' + sec) * 60,2)).split('.')[0] + ' seconds'


In [53]:
pace = min + sec

In [54]:
pace

'9 minutes, 1 seconds'

In [66]:
import pathlib

In [71]:
Path.home()

PosixPath('/home/daniel')

In [1]:
run.head()

NameError: name 'run' is not defined

In [51]:
emojis = pd.read_json('/home/daniel/Documents/datascience/fitprogress/data/emoji.json', orient='index')
em = list(emojis.index)
np.random.choice(emm, 1)[0]

'seat'

In [65]:
emojis[emojis.index.str.contains('run')]

,0
running_shirt_with_sash,🎽
woman-running,🏃‍♀️
man-running,🏃‍♂️
runner,🏃‍♂️
running,🏃‍♂️


'dove_of_peace'

In [26]:
x = round(run.avg_pace.dt.seconds / 60, 2)

In [38]:
time_mins = round(run.time.dt.seconds / 60, 2)

In [39]:
fig = px.scatter(data_frame=run, 
                        y=time_mins,   # choice 
                        hover_data=['distance', 'calories'], 
                        labels=dict(x='Date', time='Time', avg_run_cadence='Average Run Cadence', avg_pace='Average Pace'), 
                        title='Snapshot',
                        size='distance',
                        color='distance',
                        template='plotly',
                        height=500, 
                        width=800,
                        )

In [40]:
fig.show()

In [27]:
fig = px.scatter(data_frame=run,
                y='avg_run_cadence', 
                x=x,
                 )
fig.show()

In [ ]:
  fig = px.scatter(data_frame=run_slice, 
                        y=choice,   # choice 
                        hover_data=['distance', 'calories'], 
                        labels=dict(x='Date', time='Time', avg_run_cadence='Average Run Cadence', avg_pace='Average Pace'), 
                        title='Snapshot',
                        size='distance',
                        color='distance',
                        template='plotly',
                        height=700, 
                        width=1200,
                        )

In [60]:
dataraw.Time = dataraw.Time.apply(lambda x: x.split('.')[0])

In [62]:
dataraw['timedelta'] = pd.to_timedelta(dataraw.Time)

In [88]:
rawrun = dataraw[dataraw['Activity Type'] == 'Running']

In [90]:
dataraw.Time = pd.to_timedelta(dataraw.Time)

In [87]:
dataraw.columns

Index(['Activity Type', 'Date', 'Favorite', 'Title', 'Distance', 'Calories',
       'Time', 'Avg HR', 'Max HR', 'Aerobic TE', 'Avg Run Cadence',
       'Max Run Cadence', 'Avg Pace', 'Best Pace', 'Elev Gain', 'Elev Loss',
       'Avg Stride Length', 'Avg Vertical Ratio', 'Avg Vertical Oscillation',
       'Training Stress Score®', 'Grit', 'Flow', 'Climb Time', 'Bottom Time',
       'Min Temp', 'Surface Interval', 'Decompression', 'Best Lap Time',
       'Number of Laps', 'Max Temp', 'Unnamed: 0', 'Unnamed: 0.1',
       'Total Strokes', 'Total Reps', 'Total Sets', 'timedelta',
       'calories per minute'],
      dtype='object')

In [74]:
dataraw['Calories'][2] / (dataraw['timedelta'][2].seconds / 60)

17.91416705122289

In [86]:
dataraw.timedelta.dt.seconds / 60 / 60

0      0.373889
1      0.611389
2      0.601944
3      0.790833
4      0.428333
         ...   
344    0.164722
345    4.080556
346    0.485556
347    0.974167
348    1.086944
Name: timedelta, Length: 349, dtype: float64

In [77]:
dataraw['calories per minute'] = dataraw['Calories'] / (dataraw.timedelta.dt.seconds / 60)

In [81]:
dataraw.sort_values(by='calories per minute', ascending=False)

,Activity Type,Date,Favorite,Title,Distance,Calories,Time,Avg HR,Max HR,Aerobic TE,...,Best Lap Time,Number of Laps,Max Temp,Unnamed: 0,Unnamed: 0.1,Total Strokes,Total Reps,Total Sets,timedelta,calories per minute
57,Running,2020-08-15 18:57:52,False,Portland Running,1.01,142.0,00:07:37,173,181,2.4,...,00:03.33,2,0.0,53.0,51.0,0,0,0,0 days 00:07:37,18.643326
348,Running,2018-09-05 18:07:48,False,Portland Running,6.01,1205.0,01:05:13,171,197,5.0,...,00:00.00,7,0.0,344.0,342.0,0,0,0,0 days 01:05:13,18.476872
42,Running,2020-09-19 10:32:09,False,Portland Running,4.01,669.0,00:36:29,168,187,4.7,...,00:05.18,5,0.0,38.0,38.0,NaN,NaN,NaN,0 days 00:36:29,18.337140
14,Running,2020-10-27 14:07:59,False,Portland Running,5.02,854.0,00:46:44,162,176,3.9,...,00:08.66,6,0.0,NaN,NaN,NaN,NaN,NaN,0 days 00:46:44,18.273894
36,Running,2020-09-30 14:30:18,False,Portland Running,3.11,493.0,00:27:00,166,183,3.7,...,00:59.25,4,0.0,32.0,32.0,NaN,NaN,NaN,0 days 00:27:00,18.259259
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
180,Cycling,2020-02-22 10:51:52,False,Portland Cycling,40.22,614.0,02:59:27,80,159,1.7,...,01:08.15,9,0.0,176.0,174.0,0,0,0,0 days 02:59:27,3.421566
128,Cycling,2020-04-19 13:36:23,False,Portland Cycling,0.05,2.0,00:00:36,87,98,0.0,...,00:00.00,1,0.0,124.0,122.0,0,0,0,0 days 00:00:36,3.333333
150,Indoor Running,2020-03-08 19:35:20,False,Indoor Running,0.05,1.0,00:00:19,100,112,0.0,...,00:00.00,1,0.0,146.0,144.0,0,0,0,0 days 00:00:19,3.157895
334,Strength Training,2019-04-10 12:30:19,False,Strength,0.00,4.0,00:01:23,94,102,0.0,...,00:00.00,1,0.0,330.0,328.0,0,"1,070",5,0 days 00:01:23,2.891566


In [65]:
dataraw.replace('--', '0', inplace=True)
dataraw.Calories = dataraw.Calories.str.replace(',', '').astype(float)

In [67]:
dataraw['cals per minute'] = dataraw.Calories / dataraw.timedelta.

AttributeError: 'TimedeltaProperties' object has no attribute 'minute'

In [51]:
run['time'][4].minute

58

In [76]:
run.index[:1]

DatetimeIndex(['2020-11-10 15:12:40'], dtype='datetime64[ns]', name='date', freq=None)

In [84]:
run.columns

Index(['activity_type', 'title', 'distance', 'calories', 'time', 'avg_hr',
       'max_hr', 'aerobic_te', 'avg_run_cadence', 'max_run_cadence',
       'avg_pace', 'best_pace', 'elev_gain', 'elev_loss', 'avg_stride_length'],
      dtype='object')

In [104]:
run.avg_pace[2].strftime('%M:%S')

'08:53'

In [87]:
fig2 = px.scatter(run, y='avg_pace', hover_data=['distance', 'calories'])
fig2.show()

In [94]:
#y = [8, 8.5, 9, 8.5, 8, 9, 8, 8.5, 9, 10]

run_slice = run.iloc[:30]
x = run.index[:10]
y = run.avg_pace[:10].values
fig = px.scatter(data_frame=run_slice, 
                y='avg_pace', 
                hover_data=['distance', 'calories'], 
                labels=dict(avg_pace='Average Pace', date='Date')
                )

fig.show()

In [219]:
last_15_weeks = pd.DataFrame(data[data.activity_type == 'Running'].resample('W-MON').distance.sum().tail(15))

In [7]:
activity_choice_last_5 = data[data.activity_type == 'Running'].head(5)

In [16]:
activity_choice_last_5[['distance', 'calories', 'time', 'avg_pace']].copy()

,distance,calories,time,avg_pace
date,,,,
2020-11-10 15:12:40,3.11,463,00:25:42,8:16
2020-11-08 16:14:17,6.26,"1,011",00:58:11,9:17
2020-11-06 14:33:08,4.35,703,00:38:38,8:53
2020-11-04 12:16:30,4.00,638,00:35:02,8:45
2020-11-03 14:17:58,4.01,623,00:35:40,8:54


In [86]:
bike.shape[0]

178

In [66]:
# I believe you're not supposed to have mixed datatypes in a dataframe.  that being said, if we do end up splitting the biking and running activities to multiple dataframes, we'll have to calculate the pace ourselves.  who am I even talking to right now? 
# final_df[final_df.activity_type == 'Cycling'].iloc[:, 10:12] = final_df[final_df.activity_type == 'Cycling'].iloc[:, 10:12].astype(float)
# final_df[final_df.activity_type == 'Running'].iloc[:, 10:12] = final_df[final_df.activity_type == 'Running'].iloc[:, 10:12].apply(lambda x: dt.datetime.strptime(x, '%M:%S'))

# 

### Checking for days I biked multiple times.  Every day I commuted....other days I did there and back rides....so that's a fair amount

In [ ]:
# the keep=False keeps all the duplicates, instead of the first or the last dup
# Syntax 1
bike[bike.date.dt.date.duplicated(keep=False)]

# Syntax 2
#bike[bike.duplicated(subset=['date'], keep=False)].sort_index()

In [ ]:
#run.resample('D').distance.sum()
last_15_weeks = pd.DataFrame(run.resample('W-MON').distance.sum().tail(15))

run.set_index("date", inplace=True)
last_15_weeks = pd.DataFrame(run.resample("W-MON").distance.sum().tail(15))

In [52]:
run['pace_mins'] = round(run.avg_pace.dt.seconds / 60, 2)

In [70]:
run.resample('M').run_length_1_dec.value_counts()

date        run_length_1_dec
2018-09-30  1.0                 1
            2.3                 1
            5.0                 1
            6.0                 1
            11.5                1
                               ..
2020-11-30  4.0                 5
            2.3                 1
            3.1                 1
            4.4                 1
            6.3                 1
Name: run_length_1_dec, Length: 62, dtype: int64

In [96]:
pd.set_option('display.max_rows', 100)

In [58]:
run['run_length'] = round(run.distance, 0)

In [84]:
run.run_length_1_dec['2019']

date
2019-11-03 15:54:19    1.0
2019-10-19 14:26:35    2.0
2019-09-03 20:00:43    2.0
2019-08-26 20:41:18    2.0
2019-08-25 16:08:04    1.4
2019-08-24 13:06:18    2.0
2019-07-29 20:46:33    2.2
2019-07-20 10:47:28    1.0
2019-06-30 19:24:11    1.0
2019-06-18 19:04:15    1.0
2019-05-05 19:11:48    0.6
2019-04-22 13:35:55    0.8
2019-04-21 10:42:57    1.5
2019-04-15 10:24:28    1.0
2019-04-11 07:43:19    1.5
2019-03-01 16:06:38    1.0
2019-03-01 15:45:03    1.0
Name: run_length_1_dec, dtype: float64

In [82]:
run.run_length_1_dec['2020-09']

date
2020-09-30 14:30:18    3.1
2020-09-28 17:56:57    5.0
2020-09-25 18:22:29    4.0
2020-09-24 13:57:19    3.1
2020-09-22 14:25:45    3.1
2020-09-20 14:24:14    4.0
2020-09-19 10:32:09    4.0
2020-09-09 12:26:49    3.1
2020-09-05 17:03:41    6.2
2020-09-02 13:02:06    3.1
Name: run_length_1_dec, dtype: float64

In [66]:
run.run_length_1_dec.value_counts(normalize=True)

3.1     0.255814
2.0     0.186047
4.0     0.155039
3.0     0.116279
1.0     0.085271
5.0     0.054264
6.2     0.031008
1.5     0.023256
2.3     0.015504
3.3     0.007752
4.4     0.007752
0.6     0.007752
6.0     0.007752
11.5    0.007752
1.4     0.007752
2.2     0.007752
7.0     0.007752
0.8     0.007752
6.3     0.007752
Name: run_length_1_dec, dtype: float64

In [109]:
run[['distance', 'run_length', 'run_length_1_dec']].head(20)

,distance,run_length,run_length_1_dec
date,,,
2020-11-15 15:03:39,2.34,2.0,2.3
2020-11-13 16:23:46,4.01,4.0,4.0
2020-11-12 11:55:48,4.02,4.0,4.0
2020-11-10 15:12:40,3.11,3.0,3.1
2020-11-08 16:14:17,6.26,6.0,6.3
2020-11-06 14:33:08,4.35,4.0,4.4
2020-11-04 12:16:30,4.00,4.0,4.0
2020-11-03 14:17:58,4.01,4.0,4.0
2020-11-01 13:21:42,4.01,4.0,4.0


In [112]:
run.run_length_1_dec.value_counts().head(10)

3.1    33
2.0    24
4.0    20
3.0    15
1.0    11
5.0     7
6.2     4
1.5     3
2.3     2
3.3     1
Name: run_length_1_dec, dtype: int64